In [ ]:
# Importing required packages
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy.optimize import fsolve, fminbound, fmin_bfgs

In [ ]:
# Quasi-Newton unconstained optimization 
def quasi_newton_opt(x0, func, func_grad, max_iter=500, full_output=False):
    # Function to perform a quasi-Newton optimization
    # for an unconstrained problem with BFGS Hessian update

    # x0: Inital point
    # func_grad: Gradient of function
    # func_line_search: Objective function with used for line search

    B0 = np.identity(len(x0))    # initializing hessian
    iter_no = 0
    while True:
        delta_f0 = func_grad(x0, func)    # calculating gradient
        sq = np.dot(-B0, delta_f0)  # calculating search direction
        alpha = fminbound(lambda alpha: func(x0 + alpha*sq), 0, 10000)  # line search along sq
        x_new = x0 + alpha * sq # new point
        delta_f_new = func_grad(x_new, func)  # gradient at new point 

        # Checking for stationary point or max iterations:
        if max(abs(delta_f_new)) < 1E-07 or iter_no > max_iter:
            break

        delta_x = x_new - x0    # change in x
        delta_delta = delta_f_new - delta_f0    # change in gradient
        delta_B = BFGS_update(B0, delta_x, delta_delta) # BFGS hessian update

        # Updating values for next iteration
        B_new = B0 + delta_B
        B0 = B_new
        x0 = x_new
        delta_f0 = delta_f_new
        iter_no = iter_no + 1
        
    if full_output:
        return [x_new, delta_f_new, B_new, iter_no]
    return x_new

In [ ]:
def func(x):
    # Objective function definition

    # x: Evaluation point
    f = 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2 + 100*(x[2] - x[1]**2)**2 + (1 - x[1])**2
    return f

def test_func(x):
    return np.sin(x[0] + x[1]) + (x[0] - x[1])**2 - 1.5*x[0] + 2.5*x[1] + 1

def func_grad(x, func):
    # Function to calculate gradient 
    # of objective function using forward FD

    # x: Evaluation point

    step = 1E-8 # step size for FD
    delta_f = np.zeros((len(x), 1)) # initializing gradient
    for i in range(len(x)):
        x_temp = np.array(x, dtype=float)
        x_temp[i] = x[i] + step
        delta_f[i] = (func(x_temp) - func(x)) / step    # calculating gradient using forward FD
    return delta_f

# def func_grad(x):
#     # Function to calculate gradient 
#     # of objective function using forward FD

#     # x: Evaluation point

#     step = 1E-8 # step size for FD
#     delta_f = np.zeros((2, 1))
#     delta_f[0] = (func(np.array([x[0]+step, x[1]])) - func(x)) / step
#     delta_f[1] = (func(np.array([x[0], x[1]+step])) - func(x)) / step
#     return delta_f



def BFGS_update(B, delta_x, delta_delta):
    # Function to update to hessian
    # using BFGS method

    # B: initial hessian
    # delta_x: change in evaluation point
    # delta_delta: change in gradient
    
    delta_x_T = delta_x.transpose()
    delta_delta_T = delta_delta.transpose()
    delta_B = (1 + (np.dot(np.dot(delta_delta_T, B), delta_delta))/(np.dot(delta_x_T, delta_delta))) * \
    (np.dot(delta_x, delta_x_T)/np.dot(delta_x_T, delta_delta)) - \
        (np.dot(delta_x, np.dot(delta_delta_T, B)) + np.dot(np.dot(delta_delta_T, B).transpose(), delta_x_T)) / \
            (np.dot(delta_x_T, delta_delta))
    
    return delta_B

In [ ]:
x0 = np.array([[0, 0, 0]]).transpose()
[x, delta_f, B, iter_no] = quasi_newton_opt(x0, func, func_grad, max_iter=1000, full_output=True)
print(f'Optima: \n{x}\nGradient at optima: \n{delta_f}\nHessian inverse at optima: \n{B}')
print(f'Iterations: {iter_no}')

In [ ]:
x0 = np.array([0, 0, 0], dtype=float)
x = fmin_bfgs(func, x0)
print(x)